In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import pickle

import ranch

In [2]:
root_dir = "D:\metcouncil_network_rebuild"

external_dir = os.path.join(root_dir, 'data', 'external')

corrected_network_dir = os.path.join(external_dir, 'rachel_corrected_version')

In [3]:
output_dir = os.path.join(root_dir, 'data', 'interim')

## Network object

In [4]:
# network object
working_network_filename = os.path.join(root_dir,"data", "interim","step8_network.pickle")

roadway_network = pickle.load(open(working_network_filename, 'rb'))

In [5]:
# number of unique shstReferenceIds in the network
print(roadway_network.links_df.shstReferenceId.nunique())

# number of unique shstGeometryIds in the network
print(roadway_network.links_df.shstGeometryId.nunique())

1061566
556550


## match result to the existing network

In [6]:
# shst match output
conflation_df = ranch.read_shst_extraction(os.path.join(corrected_network_dir, 'shst_match'), '*.matched.geojson')

# match input with #lanes info
input_network_df = gpd.read_file(os.path.join(corrected_network_dir, "BaseNet_NoMnPASS.shp"))

2022-04-15 14:49:19, INFO: ----------start reading shst extraction data-------------
2022-04-15 14:49:19, INFO: reading shst extraction data : D:\metcouncil_network_rebuild\data\external\rachel_corrected_version\shst_match\county_0_in.out.matched.geojson
2022-04-15 14:49:38, INFO: reading shst extraction data : D:\metcouncil_network_rebuild\data\external\rachel_corrected_version\shst_match\county_10_in.out.matched.geojson
2022-04-15 14:49:41, INFO: reading shst extraction data : D:\metcouncil_network_rebuild\data\external\rachel_corrected_version\shst_match\county_11_in.out.matched.geojson
2022-04-15 14:49:42, INFO: reading shst extraction data : D:\metcouncil_network_rebuild\data\external\rachel_corrected_version\shst_match\county_12_in.out.matched.geojson
2022-04-15 14:49:44, INFO: reading shst extraction data : D:\metcouncil_network_rebuild\data\external\rachel_corrected_version\shst_match\county_13_in.out.matched.geojson
2022-04-15 14:49:45, INFO: reading shst extraction data : D:\

In [7]:
conflation_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 466407 entries, 0 to 466406
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   shstReferenceId         466407 non-null  object  
 1   shstGeometryId          466407 non-null  object  
 2   shstFromIntersectionId  466407 non-null  object  
 3   shstToIntersectionId    466407 non-null  object  
 4   referenceLength         466407 non-null  float64 
 5   gisReferenceId          466407 non-null  object  
 6   gisGeometryId           466407 non-null  object  
 7   gisTotalSegments        466407 non-null  int64   
 8   gisSegmentIndex         466407 non-null  int64   
 9   gisFromIntersectionId   466407 non-null  object  
 10  gisToIntersectionId     466407 non-null  object  
 11  startSideOfStreet       466407 non-null  object  
 12  endSideOfStreet         466407 non-null  object  
 13  sideOfStreet            466407 non-null  object  
 

In [8]:
input_network_df.columns

Index(['link_id', 'geoId', 'shape_id', 'A', 'B', 'distance', 'roadway', 'name',
       'drive', 'walk', 'bike', 'truck', 'centroid', 'mrcc_id', 'ROUTE_SYS',
       'assgngrp', 'rdclass', 'area_type', 'county', 'mpo', 'AADT', 'count_AM',
       'count_MD', 'count_PM', 'count_NT', 'count_day', 'count_year',
       'segment_id', 'managed', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT',
       'm_lanes_AM', 'm_lanes_MD', 'm_lanes_PM', 'm_lanes_NT', 'access_AM',
       'access_MD', 'access_PM', 'access_NT', 'geometry'],
      dtype='object')

In [9]:
# number of unique shstReferenceIds in the match
print(conflation_df.shstReferenceId.nunique())

# number of unique shstGeometryIds in the match
print(conflation_df.shstGeometryId.nunique())

460278
254484


In [10]:
# join conflation database with input database
info_columns = ['A', 'B', 'mrcc_id', 'ROUTE_SYS',
       'assgngrp', 'rdclass', 'area_type', 'county', 'mpo', 'AADT', 'count_AM',
       'count_MD', 'count_PM', 'count_NT', 'count_day', 'count_year', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT']

conflation_df = pd.merge(
    conflation_df.rename(columns = {'pp_a' : 'A', 'pp_b' : 'B'}),
    input_network_df[info_columns],
    how = 'left',
    on = ['A', 'B']
)

In [11]:
conflation_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 466407 entries, 0 to 466406
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   shstReferenceId         466407 non-null  object  
 1   shstGeometryId          466407 non-null  object  
 2   shstFromIntersectionId  466407 non-null  object  
 3   shstToIntersectionId    466407 non-null  object  
 4   referenceLength         466407 non-null  float64 
 5   gisReferenceId          466407 non-null  object  
 6   gisGeometryId           466407 non-null  object  
 7   gisTotalSegments        466407 non-null  int64   
 8   gisSegmentIndex         466407 non-null  int64   
 9   gisFromIntersectionId   466407 non-null  object  
 10  gisToIntersectionId     466407 non-null  object  
 11  startSideOfStreet       466407 non-null  object  
 12  endSideOfStreet         466407 non-null  object  
 13  sideOfStreet            466407 non-null  object  
 

In [12]:
# drop duplciates in the conflation database

conflation_df.drop_duplicates(inplace = True)

conflation_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 466407 entries, 0 to 466406
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   shstReferenceId         466407 non-null  object  
 1   shstGeometryId          466407 non-null  object  
 2   shstFromIntersectionId  466407 non-null  object  
 3   shstToIntersectionId    466407 non-null  object  
 4   referenceLength         466407 non-null  float64 
 5   gisReferenceId          466407 non-null  object  
 6   gisGeometryId           466407 non-null  object  
 7   gisTotalSegments        466407 non-null  int64   
 8   gisSegmentIndex         466407 non-null  int64   
 9   gisFromIntersectionId   466407 non-null  object  
 10  gisToIntersectionId     466407 non-null  object  
 11  startSideOfStreet       466407 non-null  object  
 12  endSideOfStreet         466407 non-null  object  
 13  sideOfStreet            466407 non-null  object  
 

In [13]:
# in conflation df, aggregate based on shstReferenceId, get all number of lanes for each shstReferenceId

lanes_conflation_df = conflation_df.groupby(
    ['shstReferenceId']
)['lanes_AM'].apply(list).to_frame().reset_index()

In [14]:
lanes_conflation_df['lanes_metc_min'] = lanes_conflation_df['lanes_AM'].apply(lambda x: min(set(x)))
lanes_conflation_df['lanes_metc_max'] = lanes_conflation_df['lanes_AM'].apply(lambda x: max(set(x)))

In [17]:
lanes_conflation_df[lanes_conflation_df.lanes_metc_min != lanes_conflation_df.lanes_metc_max]

,shstReferenceId,lanes_AM,lanes_metc_min,lanes_metc_max
11828,06810aa62b0daad9693810f968cdcc76,"[1, 2]",1,2
23605,0d070864eedfd1af320344abe2349bae,"[2, 1]",1,2
50134,1bc9472e5d0e7dbdc80919a5b56578ba,"[4, 3]",3,4
50373,1beafdffd0e372d05ea76cc85173c51d,"[1, 2]",1,2
52400,1d03b123cacb785d3a68b6d58601565b,"[2, 1]",1,2
52872,1d49beb4ddf780e431c402ab95ea56e1,"[1, 2]",1,2
54235,1e0fd1d83c1aa182d69c89f4c194c6ef,"[3, 3, 1]",1,3
62375,228a5642c7e03c0bbf5fb2985319d456,"[2, 1]",1,2
64363,239b43e00276ab9b760cf7b812f38c3a,"[1, 2]",1,2
65836,246ec79dd775527ebf79b7f3732ab610,"[1, 1, 2]",1,2


In [18]:
# in conflation df, aggregate based on shstReferenceId, get all number of lanes for each shstReferenceId

assgngrp_conflation_df = conflation_df.groupby(
    ['shstReferenceId']
)['assgngrp'].apply(list).to_frame().reset_index()

In [19]:
assgngrp_conflation_df['assgngrp_min'] = assgngrp_conflation_df['assgngrp'].apply(lambda x: min(set(x)))
assgngrp_conflation_df['assgngrp_max'] = assgngrp_conflation_df['assgngrp'].apply(lambda x: max(set(x)))

In [20]:
assgngrp_conflation_df[assgngrp_conflation_df.assgngrp_min != assgngrp_conflation_df.assgngrp_max]

,shstReferenceId,assgngrp,assgngrp_min,assgngrp_max
2253,0141620c9f5fa0e6f6549b237d788e44,"[50, 6]",6,50
3647,0208d346f81189fae772b6f3fd3efe4d,"[7, 6]",6,7
3731,021289209a2373c8421cfc1e164e22a2,"[6, 50]",6,50
4489,027c05354a7c94a6cbec05f0a75bc620,"[7, 5, 5, 5, 5]",5,7
6586,03a15e4d0b995769018d0bc1a0e28ca5,"[6, 50]",6,50
...,...,...,...,...
452464,fb9d3ccee3d17d659a2b7633f243d165,"[50, 6]",6,50
452656,fbb793323b0d610db995ded5e467fe2d,"[50, 6, 6]",6,50
458717,ff1842cb00d0f30003ab658e7965cf05,"[6, 7, 7]",6,7
460010,ffda6118eb7650ccfbe50f6850c41b40,"[50, 6, 50, 6, 50, 6]",6,50


In [21]:
# in conflation df, aggregate based on shstReferenceId, get all number of lanes for each shstReferenceId

rdclass_conflation_df = conflation_df.groupby(
    ['shstReferenceId']
)['rdclass'].apply(list).to_frame().reset_index()

In [22]:
rdclass_conflation_df['rdclass_min'] = rdclass_conflation_df['rdclass'].apply(lambda x: min(set(x)))
rdclass_conflation_df['rdclass_max'] = rdclass_conflation_df['rdclass'].apply(lambda x: max(set(x)))

In [23]:
rdclass_conflation_df[rdclass_conflation_df.rdclass_min != rdclass_conflation_df.rdclass_max]

,shstReferenceId,rdclass,rdclass_min,rdclass_max
2253,0141620c9f5fa0e6f6549b237d788e44,"[50, 30]",30,50
3731,021289209a2373c8421cfc1e164e22a2,"[40, 50]",40,50
4489,027c05354a7c94a6cbec05f0a75bc620,"[40, 20, 20, 20, 20]",20,40
6352,037e90f685177a3aa06479a2f97b94b5,"[40, 30]",30,40
6586,03a15e4d0b995769018d0bc1a0e28ca5,"[40, 50]",40,50
...,...,...,...,...
450778,faab5a6bb81a78607d60ca1554f6322e,"[40, 30]",30,40
452464,fb9d3ccee3d17d659a2b7633f243d165,"[50, 40]",40,50
452656,fbb793323b0d610db995ded5e467fe2d,"[50, 30, 30]",30,50
460010,ffda6118eb7650ccfbe50f6850c41b40,"[50, 20, 50, 20, 50, 20]",20,50


In [24]:
# in conflation df, aggregate based on shstReferenceId, get all number of lanes for each shstReferenceId

aadt_conflation_df = conflation_df.groupby(
    ['shstReferenceId']
)['AADT'].apply(list).to_frame().reset_index()

In [25]:
aadt_conflation_df['AADT_min'] = aadt_conflation_df['AADT'].apply(lambda x: min(set(x)))
aadt_conflation_df['AADT_max'] = aadt_conflation_df['AADT'].apply(lambda x: max(set(x)))

In [26]:
aadt_conflation_df[aadt_conflation_df.AADT_min != aadt_conflation_df.AADT_max]

,shstReferenceId,AADT,AADT_min,AADT_max
388,003321b0e2dc7f6bce892e3ee0483eb5,"[850, 0]",0,850
1804,00f95550115d2c2e5a21d2ee38203165,"[16100, 0]",0,16100
1833,00fd1b134807754961b42eb8afa8c747,"[0, 1300]",0,1300
2253,0141620c9f5fa0e6f6549b237d788e44,"[0, 3950]",0,3950
4273,025dc96a8501e67e47a36570db8499c7,"[0, 4500]",0,4500
...,...,...,...,...
452464,fb9d3ccee3d17d659a2b7633f243d165,"[0, 85]",0,85
455285,fd263acc5415c958467ff4bcea4b4f11,"[0, 7900]",0,7900
455629,fd564cb302f7eb4ca9b2287d478199ba,"[0, 1100, 0, 0]",0,1100
458717,ff1842cb00d0f30003ab658e7965cf05,"[0, 5400, 90]",0,5400


In [27]:
conflation_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 466407 entries, 0 to 466406
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   shstReferenceId         466407 non-null  object  
 1   shstGeometryId          466407 non-null  object  
 2   shstFromIntersectionId  466407 non-null  object  
 3   shstToIntersectionId    466407 non-null  object  
 4   referenceLength         466407 non-null  float64 
 5   gisReferenceId          466407 non-null  object  
 6   gisGeometryId           466407 non-null  object  
 7   gisTotalSegments        466407 non-null  int64   
 8   gisSegmentIndex         466407 non-null  int64   
 9   gisFromIntersectionId   466407 non-null  object  
 10  gisToIntersectionId     466407 non-null  object  
 11  startSideOfStreet       466407 non-null  object  
 12  endSideOfStreet         466407 non-null  object  
 13  sideOfStreet            466407 non-null  object  
 

In [28]:
conflation_df[conflation_df.shstReferenceId == '01c2ac344e5596c321ef00cfeb0b08df'][['A', 'B']]

,A,B
230506,12320,168118


In [29]:
conflation_df[conflation_df.A == 12319]

,shstReferenceId,shstGeometryId,shstFromIntersectionId,shstToIntersectionId,referenceLength,gisReferenceId,gisGeometryId,gisTotalSegments,gisSegmentIndex,gisFromIntersectionId,...,count_AM,count_MD,count_PM,count_NT,count_day,count_year,lanes_AM,lanes_MD,lanes_PM,lanes_NT
230504,0f716dc0b0c4bf7eac6c206989a8bc93,f792ab468695bd0d3206b640193154ce,8d16759bbb77cf055f0790538c205b4b,41ba8e641c6a88ecdeb2c9cf9a8da2c5,642.79,7a2a23defa41d55e6dda4485adca6144,f792ab468695bd0d3206b640193154ce,1,1,b857ae0302d7b4b8bc2ab23779210879,...,0,0,0,0,0,2017,1,1,1,1
230505,d99b0f21d445016272c5f9b6bd68396b,8e38b684bf50c770113e089ff6ae6f98,8d16759bbb77cf055f0790538c205b4b,08d50a97954e396a41d5d8d4dc8aa063,512.10,ea6764984b787a28f2562d44d291af11,8e38b684bf50c770113e089ff6ae6f98,1,1,b857ae0302d7b4b8bc2ab23779210879,...,0,0,0,0,0,2017,1,1,1,1


In [58]:
lanes_conflation_df.to_csv(os.path.join(output_dir, 'metc_reviewed_lanes.csv'), index = False)

In [59]:
assgngrp_conflation_df.to_csv(os.path.join(output_dir, 'metc_reviewed_assgngrp.csv'), index = False)

In [60]:
rdclass_conflation_df.to_csv(os.path.join(output_dir, 'metc_reviewed_rdclass.csv'), index = False)

In [61]:
aadt_conflation_df.to_csv(os.path.join(output_dir, 'metc_reviewed_aadt.csv'), index = False)

### get crosswalk of old network link id to new network link ID

In [30]:
network_link_id_df = pd.merge(
    conflation_df[['shstReferenceId', 'A', 'B']],
    input_network_df[['A', 'B', 'link_id']],
    how ='left',
    on = ['A', 'B']
)

network_link_id_df.rename(columns = {'link_id':'model_link_id_old', 'A':'A_old', 'B':'B_old'}, inplace = True)

network_link_id_df = pd.merge(
    network_link_id_df,
    roadway_network.links_df[['shstReferenceId', 'model_link_id', 'A', 'B']],
    how ='left',
    on = ['shstReferenceId']
)

network_link_id_df.rename(columns = {'model_link_id':'model_link_id_new', 'A':'A_new', 'B':'B_new'}, inplace = True)

In [31]:
conflation_df[conflation_df.A == 74642]

,shstReferenceId,shstGeometryId,shstFromIntersectionId,shstToIntersectionId,referenceLength,gisReferenceId,gisGeometryId,gisTotalSegments,gisSegmentIndex,gisFromIntersectionId,...,count_AM,count_MD,count_PM,count_NT,count_day,count_year,lanes_AM,lanes_MD,lanes_PM,lanes_NT
328163,da3aba70ff29d5e0c35bb2a32f9ea982,d64bdaec8797f603a9bd880095ccbed7,07253f5a50e8562be85e06a32c93ed10,d1559789f8dd823f8bf00086f5958fc5,6337.74,1de0dbcdea105f35a34539829562572f,adc61c11b65b03632a929839d135b82d,1,1,2a86225b4f9c4b4849cbfbadf223c13a,...,21500,21500,21500,21500,86000,2017,3,3,3,3


In [32]:
network_link_id_df

,shstReferenceId,A_old,B_old,model_link_id_old,model_link_id_new,A_new,B_new
0,c2f57cd96062b9cc5696feed9739f4b7,3153,10010,82,937127.0,200714.0,36302.0
1,e9162d6a8e1d33d5a7a5cbdce7f23266,3153,3859,83,504352.0,200714.0,200718.0
2,d592a065763a320947a181cb28f603ea,3153,165440,84,462155.0,200714.0,289953.0
3,0c93d7c2521821337f48297b7f243a5e,3153,165440,84,27498.0,289953.0,34433.0
4,c75d2fc0c7de11abc01d9deb763f720e,3165,118345,102,431150.0,259650.0,10191.0
...,...,...,...,...,...,...,...
466402,1b60035f47c097aeefea115da7eda642,170183,170181,420988,59543.0,257090.0,257088.0
466403,ece3f1efb5b15b12d3c5392554a85ea1,170183,74513,420989,512486.0,257090.0,183970.0
466404,9ef969963ce79d46547a562b5b5165ad,170184,77840,420990,343666.0,257089.0,339119.0
466405,144014db4526a260bd4e116e0c5394c6,171166,58874,421172,44128.0,361327.0,181049.0


In [33]:
network_link_id_df.to_csv(os.path.join(output_dir, 'model_network_id_crosswalk_new.csv'), index = False)

In [34]:
network_link_id_df[network_link_id_df.model_link_id_old == 406871]

,shstReferenceId,A_old,B_old,model_link_id_old,model_link_id_new,A_new,B_new
367092,c727a7883b25bfa3eb9269c81f266aa0,108522,108764,406871,945300.0,137488.0,137542.0
367093,a23776746e49f686d886904ff3ab7989,108522,108764,406871,873139.0,137542.0,134803.0


In [35]:
network_link_id_df[network_link_id_df.model_link_id_old == 406945]

,shstReferenceId,A_old,B_old,model_link_id_old,model_link_id_new,A_new,B_new
367107,988c87b079441faf6d2224729b308965,108764,108522,406945,329791.0,137542.0,137488.0


## OSM lanes processing

In [51]:
osm_lanes_df = roadway_network.links_df[['shstReferenceId', 'lanes', 'oneWay']].copy()

In [52]:
osm_lanes_df["oneWay"].fillna("", inplace = True)
osm_lanes_df["oneWay"] = osm_lanes_df["oneWay"].apply(lambda x: "NA" if x in [None, np.nan, float('nan')] else x)
osm_lanes_df["oneWay"] = osm_lanes_df["oneWay"].apply(lambda x: str(x) if type(x) == bool else x)
osm_lanes_df["oneWay"] = osm_lanes_df["oneWay"].apply(lambda x: x if type(x) == str else ','.join(map(str, x)))
osm_lanes_df["oneWay_binary"] = osm_lanes_df["oneWay"].apply(lambda x: 0 if "False" in x else 1)

In [53]:
osm_lanes_df['lanes'].isnull().sum()

0

In [54]:
osm_lanes_df['osm_lanes_min'] = osm_lanes_df['lanes'].apply(
    lambda x: min(','.join(filter(str.isdigit, x)).split(','))
)

osm_lanes_df['osm_lanes_max'] = osm_lanes_df['lanes'].apply(
    lambda x: max(','.join(filter(str.isdigit, x)).split(','))
)

In [55]:
def get_osm_lanes_min(x):
    if x.osm_lanes_min == '':
        return 0
    else:
        if x.oneWay_binary == 0:
            return round(int(x.osm_lanes_min)/2)
        else:
            return x.osm_lanes_min

def get_osm_lanes_max(x):
    if x.osm_lanes_max == '':
        return 0
    else:
        if x.oneWay_binary == 0:
            return round(int(x.osm_lanes_max)/2)
        else:
            return x.osm_lanes_max

osm_lanes_df['osm_lanes_min'] = osm_lanes_df.apply(lambda x: get_osm_lanes_min(x), axis = 1)

osm_lanes_df['osm_lanes_max'] = osm_lanes_df.apply(lambda x: get_osm_lanes_max(x), axis = 1)

In [56]:
osm_lanes_df[osm_lanes_df.lanes.str.len() > 3].tail(20)

,shstReferenceId,lanes,oneWay,oneWay_binary,osm_lanes_min,osm_lanes_max
1071773,f27b1466fb29a39d591c9e0ed1d6f459,"[nan, , , , ]","False,False,False,False,False",0,0,0
1072811,f30076991adc3e7e32f14752edd13b25,"[nan, nan, nan, nan, nan]","False,False,False,False,False",0,0,0
1073179,f3307fe846cec27f3a15b5d03a3dc77d,"[2, nan, nan, nan]","False,False,False,False",0,1,1
1077769,f56d1054778f3f643432a850be6d53f2,"[nan, nan, nan, nan]","False,False,False,False",0,0,0
1079767,f66b96249b44c973994f6a1b4e27ab92,"[nan, nan, nan, nan, nan]","False,False,False,False,False",0,0,0
1080680,f6e46a9c7943fa29c9651fbfa347d280,"[nan, nan, nan, nan]","False,False,False,False",0,0,0
1081187,f72361ba6f19085c8137840953cc2278,"[nan, nan, nan, nan, nan]","False,False,False,False,False",0,0,0
1081736,f76953ada42ad0f1d498f58a12daae04,"[nan, nan, nan, nan]","False,False,False,False",0,0,0
1083288,f830aab829c1fe29dbe6f395a069e136,"[nan, nan, nan, nan]","False,False,False,False",0,0,0
1083540,f84fc19e69f153c516861747ece62042,"[nan, nan, nan, nan]","False,False,False,False",0,0,0


In [57]:
osm_lanes_df.to_csv(os.path.join(output_dir, 'osm_lanes.csv'), index = False)

## Anoka conflation

In [64]:
# shst match output
anoka_conflation_df = ranch.read_shst_extraction(
    'D:/metcouncil_network_rebuild/data/external/CountyLanes_3-27-19/Anoka/shst_match', 
    '*.matched.geojson'
)

# match input with #lanes info
anoka_network_df = gpd.read_file(
    'D:/metcouncil_network_rebuild/data/external/CountyLanes_3-27-19/Anoka/AnokaCo_Segments_032519.geojson'
)

2022-03-20 23:02:16, INFO: ----------start reading shst extraction data-------------
2022-03-20 23:02:16, INFO: reading shst extraction data : D:/metcouncil_network_rebuild/data/external/CountyLanes_3-27-19/Anoka\shst_match\AnokaCo_Segments_032519.out.matched.geojson
2022-03-20 23:02:17, INFO: ----------finished reading shst extraction data-------------


In [65]:
anoka_network_df

,OBJECTID,Segment_ID,ROUTE,STREETNAME,FROM_,TO_,CITY,DISTRICT,LANES,LENGTH_FT,...,COST_RECLM,COST_MLO,COST_OVL,PROJ_NUM,LINK_RECON,LINK_REHAB,AADT_TYPE,AADT_YEAR,Shape_STLe,geometry
0,1,676010,CR 76,FAWN LAKE DR NE,CSAH 24,EAST BETHEL CL,EastBethel,2,2LR,11397.3,...,588102.0,358332.0,268065.0,None,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Actual,2017,11397.315021,"LINESTRING (-93.18349 45.41411, -93.18335 45.4..."
1,2,672030,CR 72,VERDIN ST NW,243RD AVE,NORTH COUNTY LINE,StFrancis,1,2LR,1333.8,...,93577.0,57016.0,42654.0,None,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Highway Doc...,Actual,2017,1333.817811,"LINESTRING (-93.32700 45.41002, -93.32700 45.4..."
2,3,672010,CR 72,243RD AVE NW,BRIDGE ST,VERDIN ST,StFrancis,1,2LR,12135.6,...,797839.0,486125.0,363666.0,None,http://gis.anokacountymn.gov/links/Highway Doc...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Average,2017,12135.646220,"LINESTRING (-93.35237 45.38706, -93.35239 45.3..."
3,4,672020,CR 72,243RD AVE NW,VERDIN ST,237TH AVE,StFrancis,1,2LR,6371.6,...,410968.0,250404.0,187325.0,None,http://gis.anokacountymn.gov/links/Highway Doc...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Actual,2017,6371.559811,"LINESTRING (-93.32700 45.41002, -93.32342 45.4..."
4,5,636020,CSAH 36,RUTGERS ST,FAWN LAKE DR,NORTH COUNTY LINE,Linwood,6,2LR,3831.1,...,247106.0,150562.0,112634.0,None,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Average,2017,3831.067506,"LINESTRING (-93.04528 45.41220, -93.04527 45.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,502,614089,CSAH 14,MAIN ST,S END OF 35W BRIDGE,700' S OF WOODLAND DR,LinoLakes,6,2LR,1243.3,...,123694.0,75367.0,56381.0,02-614-24,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Actual,2018,1243.337059,"LINESTRING (-93.07142 45.18923, -93.07140 45.1..."
502,503,614092,CSAH 14,MAIN ST,700' N OF N END OF BRIDGE,100' N OF N END OF BRIDGE,LinoLakes,10,2LR,600.0,...,51600.0,31440.0,23520.0,02-614-24,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Average,2018,599.999944,"LINESTRING (-93.07152 45.17744, -93.07155 45.1..."
503,504,614093,CSAH 14,MAIN ST,100' N OF N END OF BRIDGE,CENTERVILLE CL,LinoLakes,11,2LR,949.4,...,91710.0,55879.0,41803.0,02-614-24,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Ditches/NoD...,Average,2018,949.412786,"LINESTRING (-93.07117 45.17486, -93.07118 45.1..."
504,505,665030,CR 65,201ST AVE NW,TIGER ST,NOWTHEN BLVD,Nowthen,1,2LR,5700.0,...,304182.0,185339.0,138650.0,None,http://gis.anokacountymn.gov/links/Ditches/NoD...,http://gis.anokacountymn.gov/links/Highway Doc...,Actual,2017,5700.004973,"LINESTRING (-93.49251 45.33539, -93.49079 45.3..."
